In [1]:
import pandas as pd
import regex as re
import numpy as np

In [2]:
dialogues = pd.read_csv("rusDialogs/dialogues.tsv", sep="\t")

In [3]:
def preprocessing(dialogue):
    prepared_dialogue = re.sub("<[^>]*>", " ", dialogue)
    prepared_dialogue = re.sub("\s+", " ", prepared_dialogue)
    prepared_dialogue = prepared_dialogue.split("Пользователь")[1:]
    prepared_dialogue = list(map(lambda x: x.strip(), prepared_dialogue))
    if (prepared_dialogue[0][0] == "2"):
        prepared_dialogue = list(map(lambda x: "USER" + str(3 - int(x[0])) + x[1:], prepared_dialogue))
    else:
        prepared_dialogue = list(map(lambda x: "USER" + x, prepared_dialogue))
    return prepared_dialogue

In [4]:
def concatenate_repliques(dialogue):
    prev = -1
    for i in range(len(dialogue)):
        if (dialogue[i].startswith("USER1: ")):
            if (prev == 1):
                dialogue[i] = dialogue[i - 1] + " <NM>" + dialogue[i][6:]
                dialogue[i - 1] = ""
            prev = 1
        elif (dialogue[i].startswith("USER2: ")):
            if (prev == 2):
                dialogue[i] = dialogue[i - 1] + " <NM>" + dialogue[i][6:]
                dialogue[i - 1] = ""
            prev = 2
        else:
            print("FATAL ERROR")
            print(dialogue[i])
            return
    dialogue = list(filter(lambda x : x != "", dialogue))
    return dialogue

In [5]:
dialogues["Prepared_dialogue"] = dialogues.dialogue.apply(preprocessing)

In [6]:
dialogues["Prepared_dialogueCR"] = dialogues.Prepared_dialogue.apply(concatenate_repliques)

In [7]:
import pickle
with open("rusDialogs/prepared_dialogue.pickle", "wb") as file:
    pickle.dump(list(dialogues.Prepared_dialogue), file)

In [8]:
with open("rusDialogs/prepared_dialogueCR.pickle", "wb") as file:
    pickle.dump(list(dialogues.Prepared_dialogueCR), file)